# Plot data

In [ ]:
# Load data:
import pandas as pd
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
data_train=pd.read_excel("DST_BIL54_train.xlsx",header=None)
list_of_drive = data_train.iloc[1].values


In [ ]:
# Create new column with time variable:
list_of_drive = list_of_drive[1:]

time = np.array([2018+1/12 * i for i in range(0, len(list_of_drive))])

In [ ]:

plt.plot(time,list_of_drive)
plt.title('Drivmidler as a function of time')
plt.xlabel('Time (Years from 2018-01)')
plt.ylabel('Drivmidler i alt')
plt.savefig("plot_data.png")

Notes
- Seems to be globally increasing somewhat linearly, stagnates towards the end and seems to be a slight seasonality. 

# OLS

In [ ]:
# calculate the parameters of an OLS regression:

# stack  with a columnn of ones:
data = np.array(list_of_drive)
time_inter = np.vstack((np.ones(len(time)), time.T)).T

# Calculate the parameters:
beta = np.linalg.inv(time_inter.T @ time_inter) @ time_inter.T @ data

In [ ]:
beta

In [ ]:
# estimate the errors of the parameters:

# calculate the residuals:
residuals = data - time_inter @ beta

# calculate the variance of the residuals:
variance = residuals.T @ residuals / (len(time) - 2)

# calculate the variance of the parameters:
var_beta = variance * np.linalg.inv(time_inter.T @ time_inter)

# calculate the standard deviation of the parameters:
std_beta = np.sqrt(np.diag(var_beta))

std_beta

In [ ]:
# forcastt of the drivmidler 
x_forcast = np.array([np.max(time)+1/12*i for i in range (1, 13)])
x_forcast = np.vstack((np.ones(len(x_forcast)), x_forcast.T)).T
y_forcast = x_forcast @ beta

# calculate the confidence interval:
t = stats.t.ppf(0.90, len(time) - 2)

# calculate the vmatrix based on variance and the x_forcast matrix:
vmatrix = variance * (1+x_forcast @ np.linalg.inv(time_inter.T @ time_inter) @ x_forcast.T)
y_forcast_upper = y_forcast + t * np.sqrt(np.diag(vmatrix))
y_forcast_lower = y_forcast - t * np.sqrt(np.diag(vmatrix))


In [ ]:
x_forcast

In [ ]:
y_forcast

In [ ]:
y_forcast_lower

In [ ]:
y_forcast_upper

In [ ]:
# load the test data:

data_test=pd.read_excel("DST_BIL54_test.xlsx",header=None)
data_test = data_test.iloc[1].values
data_test = data_test[1:]

In [ ]:
# plot the forecast data with the old data:
x_values = x_forcast[:,1]
plt.plot(time,list_of_drive)
plt.plot(x_values,data_test)
plt.plot(x_values,y_forcast)
plt.fill_between(x_values,list(y_forcast_lower),list(y_forcast_upper), color='gray', alpha=0.5)
plt.legend(["Train data",'Test data','Forecast','Confidence interval 90%'])
plt.xlabel('Time (Years from 2018-01)')
plt.ylabel('Drivmidler i alt')
plt.title('Drivmidler as a function of time')
plt.savefig('forecastOLS.png')


In [ ]:
# investigate residuals:
plt.plot(time,residuals)
plt.title('Residuals as a function of time')
plt.xlabel('Time (Years from 2018-01)')
plt.ylabel('Residuals compared to a linear model')
plt.savefig('residuals.png')

# WLS

We will now use WLS to make the linear model in Eq. (1) as a local trend model, i.e., the observation
at the latest timepoint (xN = x59 = 2022.833) has weight λ0 = 1, the observation at the second
latest timepoint (xN −1 = x58 = 2022.750) has weight λ1 , the third latest observation (xN −2 = x57 =
2022.667) has weight λ2 etc.
We start by setting λ = 0.9.

In [ ]:
# Creating the Sigma matrix for WLS
lamb = 0.9
Sigma = np.zeros((len(time),len(time)))
for i in range(len(time)):
    Sigma[i,i] = 1/(lamb**(len(time)-i))


In [ ]:
# Plot diagonals as a function of time: 

plt.plot(time,1/np.diag(Sigma))
plt.xlabel('Time (Years from 2018-01)')
plt.ylabel('lambda value')
plt.title("lambdas as a function of time")
plt.savefig("WLSLambdasVsTime.png")

In [ ]:
np.sum(1/np.diag(Sigma))

In [ ]:
#calculate the parameters: 

beta_WLS = np.linalg.inv(time_inter.T @ np.linalg.inv(Sigma) @ time_inter) @ time_inter.T @ np.linalg.inv(Sigma) @ data
beta_WLS

In [ ]:
# make prediction: 
y_pred_WLS_09 = x_forcast @ beta_WLS

In [ ]:
legend = ["train_data"]
plt.plot(time,list_of_drive)
for lamb in  [0.9,0.8,0.7,0.6]:
    Sigma = np.zeros((len(time),len(time)))
    for i in range(len(time)):
        Sigma[i,i] = 1/(lamb**(len(time)-i))
    
    beta_WLS = np.linalg.inv(time_inter.T @ np.linalg.inv(Sigma) @ time_inter) @ time_inter.T @ np.linalg.inv(Sigma) @ data
    y_pred_WLS = x_forcast @ beta_WLS
    plt.plot(x_values,y_pred_WLS)
    legend.append(f"WLS_pred_lambda {lamb}")

plt.legend(legend)
plt.xlabel('Time (Years from 2018-01)')
plt.ylabel('Drivmidler i alt')
plt.savefig("WLS_preds_labdas.png")
plt.show()

In [ ]:

plt.plot(x_values,y_pred_WLS)
plt.plot(time,list_of_drive)
plt.plot(x_values,y_forcast)
plt.legend(["WLS_pred","train_data","OLS_pred"])

# 4 Iterative update and optimal lambda

## 4.1

In [ ]:
def f(j):
    return np.array([[1],[j]])

L = np.array([[1,0],[1,1]])

# Inverse of L
Linv = inv(L)

print(L)
print(f(0))

## 4.2

In [ ]:
# lambda is in both equations, but it is powered to 0, so it becomes 1.
F_1 = f(-0)@f(-0).T

h_1 = f(-0)*data[0]

print(f'F_1 =\n {F_1}\n')
print(f'h_1 = \n {h_1}\n')


## 4.3

In [ ]:
# Initialize parameters
lambda_ = 0.9

In [ ]:
# Initializing F_N and h_N
F_N = F_1
h_N = h_1

for n in range(1,10):
    F_Np1 = F_N + lambda_**n * f(-n)@f(-n).T
    h_Np1 = lambda_ * Linv@h_N + f(0)*data[n]
    
    # Update F_1 and h_1 for next iteration
    F_N = F_Np1
    h_N = h_Np1

    print(f'F_{n+1} =\n {F_N}\n')
    print(f'h_{n+1} = \n {h_N}\n')
    print("\n")    

## 4.4

In [ ]:
yhat_Np1_1month  = np.zeros(49)
yhat_Np1_6month  = np.zeros(49)
yhat_Np1_12month = np.zeros(49)

for n in range(10,59):
    F_Np1 = F_N + lambda_**n * f(-n)@f(-n).T
    h_Np1 = lambda_ * Linv@h_N + f(0)*data[n]
    
    # Theta_hat
    thetahat_Np1 = inv(F_Np1)@h_Np1 
    
    # Predictions
    yhat_Np1_1month[n-10] = (f(1).T@thetahat_Np1)[0,0]
    yhat_Np1_6month[n-10] = (f(6).T@thetahat_Np1)[0,0]
    yhat_Np1_12month[n-10] = (f(12).T@thetahat_Np1)[0,0]
    
    # Update F_1 and h_1 for next iteration
    F_N = F_Np1
    h_N = h_Np1

## 4.5

In [ ]:
# Plot
plt.scatter(time[10:59],yhat_Np1_1month, s = 0.8, c = 'green', label = '1-month forecast')
plt.scatter(time[15:59],yhat_Np1_6month[0:44], s = 0.8, c = 'orange', label = '6-month forecast')
plt.scatter(time[21:59],yhat_Np1_12month[0:38], s = 0.8, c = 'red', label = '12-month forecast')
plt.plot(time,data)
plt.title('Drivmidler as a function of time')
plt.xlabel('Time (Year)')
plt.ylabel('Drivmidler i alt')
plt.legend()

## 4.6

In [ ]:
# Creating np.array with lambdas
lambdas = np.arange(0.55,0.96,0.01)

# Initializing np.arrays for predicted values
yhat_Np1_1month  = np.zeros(49)
yhat_Np1_6month  = np.zeros(49)
yhat_Np1_12month = np.zeros(49)

# Initializing RMSE
RMSE_1month = np.zeros(len(lambdas))
RMSE_6month = np.zeros(len(lambdas))
RMSE_12month = np.zeros(len(lambdas))

for i in range(0,len(lambdas)):
    # Initializing F_N and h_N
    F_N = F_1
    h_N = h_1
    
    # Update FN and hN recursively, predictions are not calculated for the first 10 steps
    for n in range(1,10):
        F_Np1 = F_N + lambdas[i]**n * f(-n)@f(-n).T
        h_Np1 = lambdas[i] * Linv@h_N + f(0)*data[n]

        # Update F_N and h_N for next iteration
        F_N = F_Np1
        h_N = h_Np1
    
    # Update FN and hN recursively for the rest, predictions are now calculated
    for n in range(10,59):
        F_Np1 = F_N + lambdas[i]**n * f(-n)@f(-n).T
        h_Np1 = lambdas[i] * Linv@h_N + f(0)*data[n]

        # Theta_hat
        thetahat_Np1 = inv(F_Np1)@h_Np1 

        # Predictions
        yhat_Np1_1month[n-10] = (f(1).T@thetahat_Np1)[0,0]
        yhat_Np1_6month[n-10] = (f(6).T@thetahat_Np1)[0,0]
        yhat_Np1_12month[n-10] = (f(12).T@thetahat_Np1)[0,0]

        # Update F_1 and h_1 for next iteration
        F_N = F_Np1
        h_N = h_Np1
    
    # Calculating RMSE
    RMSE_1month[i]  = np.sqrt(np.mean((data[10:59] - yhat_Np1_1month)**2))
    RMSE_6month[i]  = np.sqrt(np.mean((data[15:59] - yhat_Np1_6month[0:44])**2))
    RMSE_12month[i] = np.sqrt(np.mean((data[21:59] - yhat_Np1_12month[0:38])**2))

In [ ]:
# Plot
plt.scatter(lambdas,RMSE_1month, s = 0.8, c = 'green', label = '1-month forecast')
plt.scatter(lambdas,RMSE_6month, s = 0.8, c = 'orange', label = '6-month forecast')
plt.scatter(lambdas,RMSE_12month, s = 0.8, c = 'red', label = '12-month forecast')
plt.title('RMSE as a function of different λ-values')
plt.xlabel('λ')
plt.ylabel('RMSE')
plt.ylim(0,30000)
plt.legend()

## 4.7

In [ ]:
p1m_opt_lambda = lambdas[np.argmin(RMSE_1month)]
print("Lambda giving the lowest possible RMSE for 1 month prediction is : " + str(p1m_opt_lambda))
print("With the value:" + str(np.min(RMSE_1month)))
# sanity check

F_N = F_1
h_N = h_1

for n in range(1,10):
    F_Np1 = F_N + p1m_opt_lambda**n * f(-n)@f(-n).T
    h_Np1 = p1m_opt_lambda * Linv@h_N + f(0)*data[n]

    # Update F_N and h_N for next iteration
    F_N = F_Np1
    h_N = h_Np1
    
# Update FN and hN recursively for the rest, predictions are now calculated
for n in range(10,59):
    F_Np1 = F_N + p1m_opt_lambda**n * f(-n)@f(-n).T
    h_Np1 = p1m_opt_lambda * Linv@h_N + f(0)*data[n]

    # Theta_hat
    thetahat_Np1 = inv(F_Np1)@h_Np1 

    # Predictions
    yhat_Np1_1month[n-10] = (f(1).T@thetahat_Np1)[0,0]

    # Update F_1 and h_1 for next iteration
    F_N = F_Np1
    h_N = h_Np1
    
# Calculating RMSE
RMSE_1month_sanity  = np.sqrt(np.mean((data[10:59] - yhat_Np1_1month)**2))

print("sanity check: " + str(RMSE_1month_sanity))



## 4.8

In [ ]:
p6m_opt_lambda = lambdas[np.argmin(RMSE_6month)]
print("Lambda giving the lowest possible RMSE for 6 month prediction is : " + str(p6m_opt_lambda))
print("With the value:" + str(np.min(RMSE_6month)))
# sanity check

F_N = F_1
h_N = h_1

for n in range(1,10):
    F_Np1 = F_N + p6m_opt_lambda**n * f(-n)@f(-n).T
    h_Np1 = p6m_opt_lambda * Linv@h_N + f(0)*data[n]

    # Update F_N and h_N for next iteration
    F_N = F_Np1
    h_N = h_Np1
    
# Update FN and hN recursively for the rest, predictions are now calculated
for n in range(10,59):
    F_Np1 = F_N + p6m_opt_lambda**n * f(-n)@f(-n).T
    h_Np1 = p6m_opt_lambda * Linv@h_N + f(0)*data[n]

    # Theta_hat
    thetahat_Np1 = inv(F_Np1)@h_Np1 

    # Predictions
    yhat_Np1_6month[n-10] = (f(6).T@thetahat_Np1)[0,0]

    # Update F_1 and h_1 for next iteration
    F_N = F_Np1
    h_N = h_Np1
    
# Calculating RMSE
RMSE_6month_sanity  = np.sqrt(np.mean((data[15:59] - yhat_Np1_6month[0:44])**2))

print("sanity check: " + str(RMSE_6month_sanity))

## 4.9

In [ ]:
p12m_opt_lambda = lambdas[np.argmin(RMSE_12month)]
print("Lambda giving the lowest possible RMSE for 12 month prediction is : " + str(p12m_opt_lambda))
print("With the value:" + str(np.min(RMSE_12month)))
# sanity check

F_N = F_1
h_N = h_1

for n in range(1,10):
    F_Np1 = F_N + p12m_opt_lambda**n * f(-n)@f(-n).T
    h_Np1 = p12m_opt_lambda * Linv@h_N + f(0)*data[n]

    # Update F_N and h_N for next iteration
    F_N = F_Np1
    h_N = h_Np1
    
# Update FN and hN recursively for the rest, predictions are now calculated
for n in range(10,59):
    F_Np1 = F_N + p12m_opt_lambda**n * f(-n)@f(-n).T
    h_Np1 = p12m_opt_lambda * Linv@h_N + f(0)*data[n]

    # Theta_hat
    thetahat_Np1 = inv(F_Np1)@h_Np1 

    # Predictions
    yhat_Np1_12month[n-10] = (f(12).T@thetahat_Np1)[0,0]

    # Update F_1 and h_1 for next iteration
    F_N = F_Np1
    h_N = h_Np1
    
# Calculating RMSE
RMSE_12month_sanity = np.sqrt(np.mean((data[21:59] - yhat_Np1_12month[0:38])**2))

print("sanity check: " + str(RMSE_12month_sanity))

## 4.10

In [ ]:
#computing RMSE for persistence model.

RMSE_persistence = np.sqrt(np.mean((data[10:59]-data[9:58])**2))

print("The root-minimum-square of the persistent model is: " + str(RMSE_persistence))



## 4.11

In [ ]:
# calculate hat_Y_60,65,71_given_59

# Creating np.array with lambdas
lambdas = np.array([0.55,0.78,0.95])

for i in range(0,len(lambdas)):
    # Initializing F_N and h_N
    F_N = F_1
    h_N = h_1
    
    # Update FN and hN recursively, predictions are not calculated for the first 10 steps
    for n in range(1,10):
        F_Np1 = F_N + lambdas[i]**n * f(-n)@f(-n).T
        h_Np1 = lambdas[i] * Linv@h_N + f(0)*data[n]

        # Update F_N and h_N for next iteration
        F_N = F_Np1
        h_N = h_Np1
    
    # Update FN and hN recursively for the rest, predictions are now calculated
    for n in range(10,59):
        F_Np1 = F_N + lambdas[i]**n * f(-n)@f(-n).T
        h_Np1 = lambdas[i] * Linv@h_N + f(0)*data[n]

        # Theta_hat
        thetahat_Np1 = inv(F_Np1)@h_Np1 

        # Predictions

        # Update F_1 and h_1 for next iteration
        F_N = F_Np1
        h_N = h_Np1
    
    if i == 0:
        yhat_Np1_1month = np.array([(f(1).T@thetahat_Np1)[0,0],(f(6).T@thetahat_Np1)[0,0],(f(12).T@thetahat_Np1)[0,0]])
    elif i == 1:
        yhat_Np1_6month = np.array([(f(1).T@thetahat_Np1)[0,0],(f(6).T@thetahat_Np1)[0,0],(f(12).T@thetahat_Np1)[0,0]])
    else:
        yhat_Np1_12month = np.array([(f(1).T@thetahat_Np1)[0,0],(f(6).T@thetahat_Np1)[0,0],(f(12).T@thetahat_Np1)[0,0]])

x_forcast = np.array([np.max(time)+1/12*i for i in range (1, 13)])
x_forcast = np.vstack((np.ones(len(x_forcast)), x_forcast.T)).T
x_values = x_forcast[:,1] 
plt.plot(time,list_of_drive)
plt.plot(x_values,data_test)
indices = [0, 5, 11]
threep_x_values = x_values[indices]
plt.scatter(threep_x_values, yhat_Np1_1month, color='red', marker='o', label='1 month predict, lambda=0.55')
plt.scatter(threep_x_values, yhat_Np1_6month, color='blue', marker='s', label='6 month predict, lambda=0.78')
plt.scatter(threep_x_values, yhat_Np1_12month, color='green', marker='^', label='12 month predict, lambda=0.95')
plt.legend(["Train data",'Test data','lambda=0.55','lambda=0.78','lambda=0.95'])
plt.xlabel('Time (Years from 2018-24)')
plt.ylabel('Drivmidler i alt')
plt.title('Prediction of drivmidler using iterative updates')
plt.savefig('E4_11.png')

